In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, activation=None, **kwargs):
        self.units = units
        self.activation = activation
        super(CustomLayer, self).__init__(**kwargs)
    def build(self, input_shape):
            self.w = self.add_weight(
                shape=(input_shape[-1], self.units),
                initializer="random_normal",
                trainable=True,
            )
            self.b = self.add_weight(
                shape=(self.units,), initializer="random_normal", trainable=True
            )

    def call(self, inputs):
        shift_1 = tf.roll(self.w, shift=1, axis=0)

        shift_2 = tf.roll(shift_1, shift=1, axis=0)
        if self.activation == "relu":
            combi = tf.keras.activations.relu(self.b + tf.matmul(inputs, self.w)) + tf.matmul(inputs, shift_1) + tf.matmul(inputs, shift_2)  )
        else:
            combi = tf.matmul(inputs, self.w) + tf.matmul(inputs, shift_1) + tf.matmul(inputs, shift_2)  + self.b

        return combi

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  CustomLayer(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  CustomLayer(10)
])

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 4.7495123e-04, -5.9372500e-02,  2.0051146e-01, -7.4056143e-01,
        -1.2479876e-01,  6.1855972e-01, -2.5166902e-01, -8.0648738e-01,
         1.5139848e-01,  2.5582761e-03]], dtype=float32)

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.1023967 , 0.09644829, 0.12507218, 0.04880429, 0.09034004,
        0.18998389, 0.07957584, 0.04569058, 0.11907791, 0.10261025]],
      dtype=float32)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
loss_fn(y_train[:1], predictions).numpy()


1.660816

In [ ]:
model.compile(optimizer='sgd',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.6431 - accuracy: 0.8176
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3402 - accuracy: 0.9011
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2732 - accuracy: 0.9209
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2337 - accuracy: 0.9324
Epoch 5/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2069 - accuracy: 0.9412


In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.1651 - accuracy: 0.9513 - 694ms/epoch - 2ms/step


[0.16514192521572113, 0.9513000249862671]

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
print(y_test[:5])
for row in probability_model(x_test[:5]):
    print(np.argmax(row))

[7 2 1 0 4]
7
2
1
0
4
